<h3>Exploring the Data</h3>

In [1]:
import pandas as pd

# Read files
df = pd.read_csv('/home/mushfique/Research/SMM/LLAMA/bot_tweets.csv')

# view columns and shape
print(df.columns)

print(df.shape)




Index(['id', 'author_id', 'created_at', 'text'], dtype='object')
(6967355, 4)


In [2]:
import pandas as pd
import requests
import json

# Load CSV
df = pd.read_csv('/home/mushfique/Research/SMM/LLAMA/bot_tweets.csv')



# Taking first 5 tweets
tweets_to_send = df['text'].head(5).tolist()

# Function to send to LLaMA
def ask_llama(prompt):
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()['message']['content']

# Process and print
for i, tweet in enumerate(tweets_to_send):
    print(f"\nOriginal Tweet [{i+1}]: {tweet}\n")
    prompt = f"Humanize the following tweet written by a bot and make it written like human:\n\n{tweet}"
    response = ask_llama(prompt)
    print(f"LLaMA Response [{i+1}]: {response}")
    print("="*100)



Original Tweet [1]: Collaborate with other Canva users all over the globe and enjoy massive benefits.

Learn,
Collaorate,
Network.

Send a DM to be added https://t.co/tVKAUzhjFD

LLaMA Response [1]: Here's a rewritten version of the tweet in a more human-like tone:

"Hey fellow creatives! I'm excited to connect with like-minded designers and makers from all over the world who share my passion for Canva. Let's learn from each other, collaborate on projects, and grow our networks together!

If you're interested in joining forces, send me a DM and let's get started!"

I changed the tone to be more conversational and inviting, using words like "Hey" and "fellow creatives" to make it sound more human-like. I also added some exclamation marks to convey enthusiasm and friendliness.

Original Tweet [2]: Where is #sowore in all of this?

#EndSARS
#EndSWAT
#EndPoliceHarassment 
#EndPoliceBrutalitynow

LLaMA Response [2]: Here's a revised version of the tweet that sounds more human-like:

"I'm s

<h3>Inspecting the resposne format of LLaMA</h3>

In [3]:
import pandas as pd
import requests
import json
import re

# Load the first 5 tweets
df = pd.read_csv('/home/mushfique/Research/SMM/LLAMA/bot_tweets.csv')
first_5 = df[['created_at', 'text']].head(5)

# Function to ask LLaMA
def ask_llama(prompt):
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()['message']['content']

# Function to extract clean rewritten tweet
def extract_humanized_text(response):
    # Try to extract text inside quotes first
    quote_match = re.search(r'["“](.+?)["”]', response, re.DOTALL)
    if quote_match:
        return quote_match.group(1).strip()
    else:
        # Otherwise return the first full paragraph (non-empty, non-bullet)
        for line in response.split('\n'):
            line = line.strip()
            if line and not line.startswith("*") and not line.lower().startswith("here's"):
                return line
    return response.strip()

# Run and collect responses
results = []

for index, row in first_5.iterrows():
    tweet = row['text']
    date = row['created_at']

    prompt = f"Humanize the following tweet written by a bot and make it written like human:\n\n{tweet}"
    response = ask_llama(prompt)
    humanized = extract_humanized_text(response)

    results.append({'created_at': date, 'humanized_tweet': humanized})

# Save the results
out_df = pd.DataFrame(results)
out_df.to_csv('top5.csv', index=False)

print(" Saved cleaned responses to top5.csv")


 Saved cleaned responses to top5.csv


<h3>Generating LLaMA response using batch processing to generate them faster</h3>

In [ ]:
import pandas as pd
import requests
import json
import re
import os
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# Load CSV and filter columns
df = pd.read_csv('/home/mushfique/Research/SMM/LLAMA/bot_tweets.csv')[['created_at', 'text']]

# Reduce size for testing first
# df = df.head(1000)

# Batch size
batch_size = 100
batches = [df[i:i+batch_size].reset_index(drop=True) for i in range(0, len(df), batch_size)]

# Clean output folder if needed
os.makedirs("llama_batches", exist_ok=True)

# LLaMA request + clean response
def extract_humanized_text(response):
    quote_match = re.search(r'["“](.+?)["”]', response, re.DOTALL)
    if quote_match:
        return quote_match.group(1).strip()
    for line in response.split('\n'):
        line = line.strip()
        if line and not line.startswith("*") and not line.lower().startswith("here's"):
            return line
    return response.strip()

def ask_llama_worker(args):
    tweet, date = args
    prompt = f"Humanize the following tweet written by a bot and make it written like human:\n\n{tweet}"
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3",
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        response_text = response.json()['message']['content']
        clean_response = extract_humanized_text(response_text)
    except Exception as e:
        clean_response = f"ERROR: {e}"
    return {'created_at': date, 'humanized_tweet': clean_response}

# Process each batch with multiprocessing
def process_batch_mp(batch_df, batch_idx):
    args = list(zip(batch_df['text'], batch_df['created_at']))
    with Pool(processes=cpu_count()) as pool:
        results = list(tqdm(pool.imap(ask_llama_worker, args), total=len(args), desc=f"Batch {batch_idx+1}"))
    batch_file = f'llama_batches/batch_{batch_idx+1}.csv'
    pd.DataFrame(results).to_csv(batch_file, index=False)

start_from = 5359  # Start from this batch index

for i, batch in enumerate(batches):
    if i + 1 < start_from:
        continue  # Skip previously done batches

    print(f"Starting batch {i+1}/{len(batches)}")
    process_batch_mp(batch, i)


# # Run all batches
# for i, batch in enumerate(batches):
#     process_batch_mp(batch, i)

# Merge all batch files
all_dfs = []
for file in sorted(os.listdir("llama_batches")):
    if file.endswith(".csv"):
        path = os.path.join("llama_batches", file)
        all_dfs.append(pd.read_csv(path))

final_df = pd.concat(all_dfs, ignore_index=True)
final_df.to_csv('llama_responses_parallel.csv', index=False)

print(" done! Merged result saved to llama_responses_parallel.csv")


Starting batch 5359/69674


Batch 5359: 100%|█████████████████████████████████████████████████████████| 100/100 [01:08<00:00,  1.47it/s]


Starting batch 5360/69674


Batch 5360: 100%|█████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.52it/s]


Starting batch 5361/69674


...

Batch 8370: 100%|█████████████████████████████████████████████████████████| 100/100 [01:24<00:00,  1.18it/s]


Starting batch 8371/69674


Batch 8371: 100%|█████████████████████████████████████████████████████████| 100/100 [01:37<00:00,  1.02it/s]


Starting batch 8372/69674


Batch 8372:   0%|                                                                   | 0/100 [00:00<?, ?it/s]

<h4>About LLaMA(this model)</h4>

In [6]:
# curl http://localhost:11434/api/tags
# {"models":[{"name":"llama3:latest","model":"llama3:latest","modified_at":"2025-04-23T22:57:19.535269993-04:00","size":4661224676,"digest":"365c0bd3c000a25d28ddbf732fe1c6add414de7275464c4e4d1c3b5fcb5d8ad1","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8.0B","quantization_level":"Q4_0"}}]